<a href="https://colab.research.google.com/github/juliowatanabe/Alura_Google_AI/blob/main/Alura_Google_AI_Taroh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este programa ...

In [144]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

In [145]:
# Configurações iniciais
import google.generativeai as genai
import numpy as np
import pandas as pd
import random
# Importa o módulo userdata do google.colab para acessar o meu API_Key armazenado no Secrets
from google.colab import userdata

api_key = userdata.get("JKW_API_Key")

GOOGLE_API_KEY= api_key
genai.configure(api_key=GOOGLE_API_KEY)

In [146]:
# Listando os modelos disponíveis
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [147]:
generation_config = {
  #"candidate_count": 1,
  "temperature": 0.5,
}

**candidate_count: 1:**

Este par chave-valor define a opção candidate_count como 1. Isso significa que o modelo irá gerar apenas uma resposta candidata para o prompt ou consulta fornecida.

**temperature: 0.5:**

Este par chave-valor define a opção temperature como 0.5. A temperatura é um parâmetro que controla a aleatoriedade ou criatividade do texto gerado. Veja como ela afeta o resultado:

Temperatura baixa (ex.: 0.0): O modelo priorizará as respostas mais prováveis e seguras, podendo gerar saídas menos criativas, mas previsíveis.
Temperatura alta (ex.: 1.0): O modelo explorará uma gama maior de possibilidades, podendo gerar saídas mais criativas ou surpreendentes, mas também aumentando o risco de respostas sem sentido.
Uma temperatura de 0.5 encontra um equilíbrio entre esses dois extremos, visando uma resposta um tanto criativa, porém coerente.

In [148]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [149]:
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [150]:
carta = random.randint(1, 21)
response = model.generate_content(f"qual é a carta {carta} do Taroh.")
response.text

'A carta 20 do Tarô é **O Julgamento**. \n'

In [151]:
significado = model.generate_content(f'Consulte e resuma o significado da Carta do Taroh {response}')

In [152]:
significado.text

'A carta 20 do Tarô é **O Julgamento**. \n'

In [153]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [154]:
title = "Taroh"
sample_text = significado.text

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [-0.0034917067, -0.057399236, -0.053332813, -0.00040706832, 0.041486327, -0.021897387, -0.014916752, 0.023526888, 0.00068992365, 0.08753618, -0.051769044, -0.012792771, -0.08497143, -0.011905183, 0.0247106, 0.012203619, -0.043571014, 0.017867351, 0.0020216086, -0.035433955, -0.0107447775, 0.0013005731, -0.06740404, 0.041429795, 0.020774929, -0.015000385, 0.00022580803, -0.0069702053, 0.006441518, 0.01239703, -0.03531922, -5.151205e-06, -0.06575413, 0.02518062, 0.058118794, -0.028383678, 0.02808756, 0.04781848, -0.01265035, 0.018016217, 0.012825008, 0.031286325, -0.017247763, 0.053464, 0.0006651578, -0.04146593, -0.009333454, 0.0030487988, 0.0029057146, -0.041597974, -0.010116136, 0.004069406, 0.060002804, -0.009207841, -0.009773253, -0.044554133, 0.024496242, 0.01343076, -0.048653957, 0.011850864, 0.00498158, 0.040801987, 0.022270095, 0.06792771, -0.041229192, 0.0068504633, -0.007572923, -0.0034949144, 0.038256846, 0.02187193, -0.006475939, -0.03064498, 0.01419819, 0.0186

In [155]:
DOCUMENT1 = {
    "Título": "Taroh",
    "Conteúdo": significado.text}

documents = [DOCUMENT1]

In [156]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Taroh,A carta 20 do Tarô é **O Julgamento**. \n


In [157]:
model = "models/embedding-001"

In [158]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [159]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Taroh,A carta 20 do Tarô é **O Julgamento**. \n,"[-0.0034917067, -0.057399236, -0.053332813, -0..."


In [160]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [161]:
consulta = "Resuma o siginificado"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

A carta 20 do Tarô é **O Julgamento**. 



In [165]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [166]:
prompt = f"Resuma esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**O Julgamento: A hora da verdade!**

É hora de encarar a música, galera! A carta 20 do Tarô, "O Julgamento", tá aqui pra te dar um toque: é hora de dar uma olhada pra dentro e ver o que tá rolando.

Não é um julgamento externo, tipo "você é culpado!", mas um momento de reflexão e autoavaliação. É como se o universo estivesse te dizendo: "Bora, dá uma geral na sua vida e vê o que precisa ser ajustado."

Então, prepare-se pra encarar seus medos, reconhecer seus erros e fazer as pazes com o passado. É hora de deixar pra trás o que não te serve mais e abraçar um novo capítulo com mais clareza e propósito.
